## 前言
kd树（k-dimensional tree）是一个包含空间信息的二项树数据结构，它是用来计算 kNN 的一个非常常用的工具。如果特征的维度是$D$，样本的数量是$N$，那么一般来讲 kd 树算法的复杂度是$O(DlogN)$，相比于穷算的$O(DN)$省去了非常多的计算量。

Python的`scikit-learn`机器学习包提供了`蛮算`、`kd树`和`ball树`三种 kNN 算法，学完本篇的读者若无兴趣自撰算法，可以非常轻松地使用该包，详细可见 scikit-learn 之 kNN 分类。

## 直觉
给定一堆已有的样本数据，和一个被询问的数据点（红色五角星），我们如何找到离五角星最近的15个点？

<img src="images/03.png" style="width:600px;"/>

先忽略在编程上的实现，想一想一个人如何主观地执行。嗯，他一定会把在五角附近的一些点中，分别计算每一个的距离，然后选最近的15个。这样可能只需要进行二三十次距离计算，而不是300次。

<img src="images/04.png" style="width:600px;"/>

如图，只对紫圈里的点进行计算。

啊哈！问题来了。我们讲到的“附近”已经包含了距离的概念，如果不经过计算我们怎么知道哪个点是在五角星的“附近”？为什么我们一下就认出了“附近”而计算机做不到？那是因为我们在观看这张图片时，得到的输入是已经带有距离概念的影像，然而计算机在进行计算时得到的则是没有距离概念的坐标数据。如果要让一个人人为地从300组坐标里选出最近的15个，而不给他图像，那么他也省不了功夫，必须要把300个全部计算一遍才行。

这样来说，我们要做的就是在干巴巴的坐标数据上进行加工，将空间分割成小块，并以合理地方法将信息进行储存，这样方便我们读取“附近”的点。

## 切割
这只危险的兔子，它又回来了！它今天上了四个纹身，爱心、月牙、星星和眼泪，下面是它的照片。

<img src="images/05.jpg" style="width:500px;"/>

我们来回答一个简单的问题：在这幅照片上，距离爱心最近的纹身是什么？记得上一篇文章中，我们选用的特征是每一只兔子的身高和体重；这次就不一样了，在这个问题中，每个纹身的特征是照片平面上的横轴和竖轴的坐标。

对于这个问题，如果进行蛮算的办法我们需要计算 3 次距离（分别和月亮、眼泪和星星算一次）。下面我们要做的是把整个空间按照左右和上下进行等分，并且把分割后的小空间以二叉树形式进行记录，这样可以很快地读取邻近的点而省去计算量。

好，我们先竖向沿中间把这个兔子切成两半

<img src="images/06.jpg" style="width:500px;"/>

再沿横向从中间切成四份

<img src="images/07.jpg" style="width:600px;"/>

再沿着竖向平分八份

<img src="images/08.jpg" style="width:700px;"/>

最后再沿横向切一次。这次有些区域是完全空白的，我们就把它舍弃不要了，得到 14 份：

<img src="images/09.jpg" style="width:800px;"/>

我们再按照上下左右的关系把切开的图片做成一个二叉树，树的每一个节点是一幅图，它的两个枝是这幅图平分出来的子图。

<img src="images/10.jpg" style="width:700px;"/>

可以看出这个树状结构包含了很多局部性的信息，因为它的每一个节点都是按照上下或者左右进行平分的，因此如果两个点在树中的距离较近，那么它们的实际距离就是比较近的。

## 搜寻
接下来我们要通过这棵二叉树找到离爱心最近的纹身。

首先从树的最顶端开始，向下搜寻找到最底部包含爱心的节点。这个操作非常简单，因为每一次分割要么是沿着某纵线$x=a$要么是沿着横线$y=a$，因此只需要判断爱心的$x$或$y$轴坐标是大于$a$还是小于$a$，便知道是向左还是右边选择树枝。

<img src="images/11.jpg" style="width:700px;"/>

在找到了爱心之后，我们沿着相同的路径向上攀爬。只爬了一节就发现了屁股上的两个纹身

<img src="images/12.png" style="width:150px;"/>

这里看出，在8平分的情况下，爱心和月亮是在同一个区域的。在某种意义上来讲它们是“近”的，但是我们还不能确定它们是最近的，因此还要继续向上攀爬寻找。再继续向上爬两个节点，都没有出现爱心和月亮以外的纹身。在下面这个节点中

<img src="images/13.png" style="width:200px;"/>

我们发现爱心和月亮之间的距离（红线）要小于爱心和分割线的距离（蓝线），也就是说，不论分割线的右边是什么情况，那边的纹身都不可能离爱心更近。因此可以判断，离爱心最近的图形是月亮。

这样，我们只计算了一次爱心和月亮之间的距离和一次爱心和分割线之间的距离，而不是分别计算爱心和其他三个纹身的距离。并且，要知道，爱心和分割线之间距离的计算非常简单，就是爱心的$x$坐标和分割线的$x$坐标的差（的绝对值），相比于计算两点之间的距离

+ $\sqrt{(x1−y1)^2+(x2−y2)^2}$

要省下很多计算量。

## 麻烦
啊，但也有可能这个搜寻最近点的过程没那么顺利。在上面的计算中，在找到了离爱心比较近的月亮之后，我们发现爱心距离分割线的距离比较远，因此确定月亮的确就是最近的。但是，在分割线的另一边有一个更近的纹身，那么情况就稍微复杂了。

就说这个兔子啊，又去加了两个纹身，一片叶子和一个圆圈。

<img src="images/14.png" style="width:500px;"/>

二叉树分割上也相应地多出这两个纹身。我们想找到离爱心最近的纹身，所以依旧向下搜寻先找到爱心。

<img src="images/15.jpg" style="width:700px;"/>

我们找来一张纸，记下在已访问节点中距离爱心最近的纹身和所对应的距离。现在这张纸还是空的。

<img src="images/16.png" style="width:700px;"/>

向上爬了一节，发现那一节的另一个枝里有月亮，于是跑下去查看月亮的坐标，计算爱心和月亮的距离，并在纸上记录$(图形=月亮，距离=d_1)$。

再回到蓝圈的节点向上爬，继续向上爬。我们发现，$d_1$（红线）大于爱心和分割线的距离（蓝线）。

<img src="images/17.png" style="width:200px;"/>

也就是说分割线的另一边可能有更近的点，所以从另一个分枝开始向下搜，找到…

<img src="images/18.png" style="width:700px;"/>

在另一个分枝中我们追溯到圆圈，并计算它与爱心的距离$d_2$，发现$d_2>d_1$，比月亮远，所以丢弃不要。

再向上爬一个节，我们发现 $d_1$（红线）大于爱心和切分线之间的距离（蓝线）

<img src="images/19.png" style="width:200px;"/>

因此，切分线的另一端可能有更近的纹身，因此我们从另一个树枝向下搜索…

<img src="images/20.png" style="width:700px;"/>

找到了叶子。（所幸在这个分枝里只搜索到了叶子，如果有更多的图形的话，还需要进行多层的递归。具体的过程会在后面的详细篇中讲解。）计算叶子和爱心之间的距离，得$d_3$，并发现$d_3<d_1$，比月亮更近，于是更新纸上的记录为 $(纹身=叶子，距离=d_3)$。

再向上攀登一节，我们发现$d_3$小于爱心和切分线的距离，因此另一边的数据就不用考虑了。

<img src="images/21.png" style="width:200px;"/>

这次我们已经爬到了树的最顶端，完成了搜索，纸上记载的$(叶子，d_3)$就是最近的纹身和对应的距离。

<img src="images/22.png" style="width:700px;"/>

## KD 树的结构
kd树是一个二叉树结构，它的每一个节点记载了`特征坐标`，`切分轴`，`指向左枝的指针`，`指向右枝的指针`。

其中，特征坐标是线性空间$\mathbb R$中的一个点$(x_1, x_2, …, x_n)$。

切分轴由一个整数$r$表示，这里$1≤r≤n$，是我们在$n$维空间中沿第$r$维进行一次分割。

节点的左枝和右枝分别都是 kd 树，并且满足：如果$y$是左枝的一个特征坐标，那么$y_r≤x_r$；并且如果$z$是右枝的一个特征坐标，那么$z_r≥x_r$。

给定一个数据样本集$S⊆R^n$和切分轴$r$，以下递归算法将构建一个基于该数据集的 kd 树，每一次循环制作一个节点：
+ 如果$|S|=1$，记录$S$中唯一的一个点为当前节点的特征数据，并且不设左枝和右枝。（$|S|$指集合$S$中元素的数量）
+ 如果$|S|>1$：
    - 将$S$内所有点按照第$r$个坐标的大小进行排序
    - 选出该排列后的中位元素（如果一共有偶数个元素，则选择中位左边或右边的元素，左或右并无影响），作为当前节点的特征坐标，并且记录切分轴$r$
    - 将$S_L$设为在$S$中所有排列在中位元素之前的元素；$S_R$设为在$S$中所有排列在中位元素后的元素
    - 当前节点的左枝设为以$S_L$为数据集并且$r$为切分轴制作出的 kd 树；当前节点的右枝设为以$S_R$为数据集并且$r$为切分轴制作出的 kd 树。再设$r ← (r+1)\ \ mod \ \ n$。（这里，我们想轮流沿着每一个维度进行分割；$mod\ \ n$是因为一共有$n$个维度，在沿着最后一个维度进行分割之后再重新回到第一个维度）

## 构造KD树的例子
上面抽象的定义和算法确实是很不好理解，举一个例子会清楚很多。首先随机在$\mathbb R^2$中随机生成13个点作为我们的数据集。起始的切分轴$r=0$；这里$r=0$对应$x$轴，而$r=1$对应$y$轴。

<img src="images/23.png" style="width:500px;"/>

首先先沿$x$坐标进行切分，我们选出$x$坐标的中位点，获取最根部节点的坐标

<img src="images/24.png" style="width:200px;"/>

并且按照该点的x坐标将空间进行切分，所有$x$坐标小于 6.27的数据用于构建左枝，$x$坐标大于6.27 的点用于构建右枝。

<img src="images/25.png" style="width:500px;"/>

在下一步中$r=0+1=1 \ \ mod \ \ 2$对应$y$轴，左右两边再按照$y$轴的排序进行切分，中位点记载于左右枝的节点。得到下面的树，左边的$x$是指这该层的节点都是沿$x$轴进行分割的。

<img src="images/26.png" style="width:300px;"/>

空间的切分如下

<img src="images/27.png" style="width:500px;"/>

下一步中$r≡1+1≡0 \ \ mod \ \ 2$，对应$x$轴，所以下面再按照$x2$坐标进行排序和切分，有

<img src="images/28.png" style="width:600px;"/>

<img src="images/29.png" style="width:500px;"/>

最后每一部分都只剩一个点，将他们记在最底部的节点中。因为不再有未被记录的点，所以不再进行切分。

<img src="images/30.png" style="width:700px;"/>

<img src="images/31.png" style="width:500px;"/>

就此完成了 kd 树的构造。

## KD 树上的 KNN 算法
给定一个构建于一个样本集的 kd 树，下面的算法可以寻找距离某个点$p$最近的$k$个样本。

+ **0**. 设$L$为一个有$k$个空位的列表，用于保存已搜寻到的最近点
+ **1**. 根据$p$的坐标值和每个节点的切分向下搜索（也就是说，如果树的节点是按照$x_r=a$进行切分，并且$p$的$r$坐标小于$a$，则向左枝进行搜索；反之则走右枝）
+ **2**. 当达到一个底部节点时，将其标记为访问过。如果$L$里不足$k$个点，则将当前节点的特征坐标加入$L$；如果$L$不为空并且当前节点的特征与$p$的距离小于$L$里最长的距离，则用当前特征替换掉$L$中离$p$最远的点
+ **3**. 如果当前节点不是整棵树最顶端节点，执行 (`a`)；反之，输出$L$，算法完成   
    + **a**. 向上爬一个节点。如果当前（向上爬之后的）节点未曾被访问过，将其标记为被访问过，然后执行 (`a1`) 和 (`a2`)；如果当前节点被访问过，再次执行 (`a`)。
        + **a1**. 如果此时$L$里不足$k$个点，则将节点特征加入$L$；如果$L$中已满$k$个点，且当前节点与$p$的距离小于$L$里最长的距离，则用节点特征替换掉$L$中离最远的点
        + **a2**. 计算$p$和当前节点切分线的距离。如果该距离大于等于$L$中距离$p$最远的距离并且$L$中已有$k$个点，则在切分线另一边不会有更近的点，执行(`3`)；如果该距离小于$L$中最远的距离或者$L$中不足$k$个点，则切分线另一边可能有更近的点，因此在当前节点的另一个枝从(`1`)开始执行

## 例子
设我们想查询的点为$p=(−1,−5)$，设距离函数是普通的$L_2$距离，我们想找距离问题点最近的$k=3$个点。如下：

<img src="images/32.png" style="width:600px;"/>

首先执行(`1`)，我们按照切分找到最底部节点。首先，我们在顶部开始：

<img src="images/33.png" style="width:600px;"/>

和这个节点的$x$轴比较一下，

<img src="images/34.png" style="width:600px;"/>

$p$的$x$轴更小。因此我们向左枝进行搜索：

<img src="images/35.png" style="width:600px;"/>

这次对比$y$轴，

<img src="images/36.png" style="width:600px;"/>

$p$的$y$值更小，因此向左枝进行搜索：

<img src="images/37.png" style="width:600px;"/>

这个节点只有一个子枝，就不需要对比了。由此找到了最底部的节点$(−4.6,−10.55)：

<img src="images/38.png" style="width:600px;"/>

在二维图上是

<img src="images/39.png" style="width:600px;"/>

此时我们执行 (`2`)。将当前结点标记为访问过，并记录下$L=[(−4.6,−10.55)]$，访问过的节点就在二叉树上显示为被划掉的好了。

然后执行 (`3`)，嗯，不是最顶端节点。好，执行 (`a`)，我爬。上面的是 $(−6.88,−5.4)$。

<img src="images/40.png" style="width:600px;"/>

<img src="images/41.png" style="width:600px;"/>

执行 (`a1`)，因为我们记录下的点只有一个，小于$k=3$，所以也将当前节点记录下，有$L=[(−4.6,−10.55),(−6.88,−5.4)]$。再执行 (`a2`)，因为当前节点的左枝是空的，所以直接跳过，回到步骤 (`3`)。(`3`) 看了一眼，好，不是顶部，交给你了，(`a`)。于是乎 (`a`) 又往上爬了一节。

<img src="images/42.png" style="width:600px;"/>

<img src="images/43.png" style="width:600px;"/>

(`a1`) 说，由于还是不够三个点，于是将当前点也记录下，有$L=[(−4.6,−10.55),(−6.88,−5.4),(1.24,−2.86)]$。当然，当前结点变为被访问过的。

(`a2`) 又发现，当前节点有其他的分枝，并且经计算得出$p$点和$L$中的三个点的距离分别是$6.62,5.89,3.10$，但是$p$和当前节点的分割线的距离只有$2.14$，小于与$L$的最大距离：

<img src="images/44.png" style="width:600px;"/>

因此，在分割线的另一端可能有更近的点。于是我们在当前结点的另一个分枝从头执行 (`1`)。好，我们在红线这里：

<img src="images/45.png" style="width:600px;"/>

要用$p$和这个节点比较$x$坐标:

<img src="images/46.png" style="width:600px;"/>

$p$的$x$坐标更大，因此探索右枝$(1.75,12.26$，并且发现右枝已经是最底部节点，因此启动 (`2`)。

<img src="images/47.png" style="width:600px;"/>

经计算，$(1.75,12.26)$与$p$的距离是$17.48$，要大于$p$与$L$的距离，因此我们不将其放入记录中。

<img src="images/48.png" style="width:600px;"/>

然后 (`3`) 判断出不是顶端节点，呼出 (`a`)，爬。

<img src="images/49.png" style="width:600px;"/>

(`a1`) 出来一算，这个节点与$p$的距离是$4.91$，要小于$p$与$L$的最大距离$6.62$

<img src="images/50.png" style="width:600px;"/>

因此，我们用这个新的节点替代$L$中离$p$最远的$(−4.6,−10.55)$。

<img src="images/51.png" style="width:600px;"/>

然后 (`a2`) 又来了，我们比对$p$和当前节点的分割线的距离

<img src="images/53.png" style="width:600px;"/>

这个距离小于$L$与$p$的最小距离，因此我们要到当前节点的另一个枝执行 (`1`)。当然，那个枝只有一个点，直接到 (`2`)。

<img src="images/54.png" style="width:600px;"/>

计算距离发现这个点离$p$比$L$更远，因此不进行替代。

<img src="images/55.png" style="width:600px;"/>

(`3`) 发现不是顶点，所以呼出 (`a`)。我们向上爬，

<img src="images/56.png" style="width:600px;"/>

这个是已经访问过的了，所以再来（`a`），

<img src="images/57.png" style="width:600px;"/>

好，（`a`）再爬，

<img src="images/58.png" style="width:600px;"/>

啊！到顶点了。所以完了吗？当然不，还没轮到 (`3`) 呢。现在是 (`a1`) 的回合。

我们进行计算比对发现顶端节点与$p$的距离比$L$还要更远，因此不进行更新。

<img src="images/59.png" style="width:600px;"/>

然后是 (`a2`)，计算$p$和分割线的距离发现也是更远。

<img src="images/60.png" style="width:600px;"/>

因此也不需要检查另一个分枝。

然后执行 (`3`)，判断当前节点是顶点，因此计算完成！输出距离$p$最近的三个样本是$L=[(−6.88,−5.4),(1.24,−2.86),(−2.96,−2.5)]$。

## 结语
kd 树的 kNN 算法节约了很大的计算量（虽然这点在少量数据上很难体现），但在理解上偏于复杂，希望本篇中的实例可以让读者清晰地理解这个算法。喜欢动手的读者可以尝试自己用代码实现 kd 树算法，但也可以用现成的机器学习包 scikit-learn 来进行计算。